### Importing Libraries 

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf 

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


### Get data from Yahoo Finance

You can change the 2 stocks as well as the market with the tickers of your choice. For some metrics you will need to use except the stock itself, a market index as a benchmark. The most common practice for major stocks is to use the S&P 500 Index. 

At the end we will have a dataframe with the returns of the 2 assets and the market, since all the risk metrics are using the returns and not the raw values.

In [ ]:
symbol_1 = 'AAPL'
symbol_2 = 'TSLA'
market = '^GSPC'

df_asset1 = yf.download(symbol_1, start='2023-01-01', end='2023-12-31')['Adj Close'].rename(symbol_1);
df_asset2 = yf.download(symbol_2, start='2023-01-01', end='2023-12-31')['Adj Close'].rename(symbol_2);
df_market = yf.download(market, start='2023-01-01', end='2023-12-31')['Close'].rename(market);
df_main = pd.merge(df_asset1, df_asset2, on='Date')
df_main = pd.merge(df_main, df_market, on='Date')
df_returns = df_main.pct_change().dropna()
df_returns

## Standard Deviation

Standard deviation indicates the degree of variation or volatility in the returns of each stock or index. A higher standard deviation implies higher volatility, while a lower standard deviation suggests lower volatility. Being a risk measure means that higher values represent a potentially riskier investment.

In [4]:
df = df_returns.copy()
    
def calculate_standard_deviation(returns):
    return np.std(returns)

print(f'{symbol_1} has a Standard Deviation of {calculate_standard_deviation(df[symbol_1])}')
print(f'{symbol_2} has a Standard Deviation of {calculate_standard_deviation(df[symbol_2])}')
print(f'{market} has a Standard Deviation of {calculate_standard_deviation(df[market])}')

AAPL has a Standard Deviation of 0.012544777745598272
TSLA has a Standard Deviation of 0.03309894573475674
^GSPC has a Standard Deviation of 0.008241846331439819


## Beta

Beta measures the sensitivity of a stock's returns to changes in the returns of a benchmark index. A beta greater than 1 indicates higher volatility compared to the benchmark, while a beta less than 1 suggests lower volatility.

In [5]:
df = df_returns.copy()
    
def calculate_beta(returns, benchmark_returns):
    covariance = returns.cov(benchmark_returns)
    variance = benchmark_returns.var()
    beta = covariance / variance
    return beta

print(f'{symbol_1} has a Beta of {calculate_beta(df[symbol_1], df[market])} using {market} as the benchmark')
print(f'{symbol_2} has a Beta of {calculate_beta(df[symbol_2], df[market])} using {market} as the benchmark')


AAPL has a Beta of 1.1045133228679587 using ^GSPC as the benchmark
TSLA has a Beta of 2.219296862746592 using ^GSPC as the benchmark


## Maximum Drawdown:

Maximum Drawdown is a risk metric that measures the maximum loss from a peak to a trough of a portfolio's value during a specific period, usually expressed as a percentage. It provides insight into the worst-case scenario or the largest decline an investment experienced over a certain time frame.


In [6]:
df = df_returns.copy()
    
def calculate_max_drawdown(returns):
    wealth_index = np.cumprod(1 + returns)
    peak_index = np.argmax(wealth_index)
    trough_index = np.argmin(wealth_index[:peak_index + 1])

    max_drawdown = (wealth_index.iloc[trough_index] - wealth_index.iloc[peak_index]) / wealth_index.iloc[peak_index]
    return max_drawdown

print(f'{symbol_1} has a maximum drawdown of {calculate_max_drawdown(df[symbol_1])}')
print(f'{symbol_2} has a maximum drawdown of {calculate_max_drawdown(df[symbol_2])}')
print(f'{market} has a maximum drawdown of {calculate_max_drawdown(df[market])}')



AAPL has a maximum drawdown of -0.37244419520189614
TSLA has a maximum drawdown of -0.6238494657551136
^GSPC has a maximum drawdown of -0.2038843028608435


## Sharpe Ratio:

The Sharpe ratio is a measure of risk-adjusted performance and indicates the excess return generated per unit of risk, with risk measured as the standard deviation of returns. Higher Sharpe ratios generally suggest better risk-adjusted performance

In [7]:
df = df_returns.copy()
    
def calculate_sharpe_ratio(returns, risk_free_rate):
    average_return = np.mean(returns)
    volatility = np.std(returns)
    sharpe_ratio = (average_return - risk_free_rate) / volatility
    return sharpe_ratio

print(f'{symbol_1} has a Sharpe ratio of {calculate_sharpe_ratio(df[symbol_1], 0)}')
print(f'{symbol_2} has a Sharpe ratio of {calculate_sharpe_ratio(df[symbol_2], 0)}')
print(f'{market} has a Sharpe ratio of {calculate_sharpe_ratio(df[market], 0)}')

AAPL has a Sharpe ratio of 0.14627021721972275
TSLA has a Sharpe ratio of 0.11760865791443038
^GSPC has a Sharpe ratio of 0.11184268785503289


## Sortino Ratio: 

Similar to the Sharpe ratio, the Sortino ratio focuses on downside volatility, considering only the standard deviation of negative returns. It provides a measure of risk-adjusted return with a focus on downside risk.

In [8]:
df = df_returns.copy()
    
def calculate_sortino_ratio(returns, risk_free_rate=0):
    downside_returns = np.minimum(returns - risk_free_rate, 0)
    downside_volatility = np.std(downside_returns)

    average_return = np.mean(returns - risk_free_rate)  # Adjust for risk-free rate

    if downside_volatility == 0:
        return np.inf  # Avoid division by zero

    sortino_ratio = average_return / downside_volatility
    return sortino_ratio

print(f'{symbol_1} has a Sortino ratio of {calculate_sortino_ratio(df[symbol_1])}')
print(f'{symbol_2} has a Sortino ratio of {calculate_sortino_ratio(df[symbol_2])}')
print(f'{market} has a Sortino ratio of {calculate_sortino_ratio(df[market])}')

AAPL has a Sortino ratio of 0.2703069618656741
TSLA has a Sortino ratio of 0.2198285494487039
^GSPC has a Sortino ratio of 0.20205301637520193


## Treynor Ratio: 

The Treynor Ratio is another risk-adjusted performance measure similar to the Sharpe ratio. It's calculated by dividing the excess return over the risk-free rate by the beta of the investment, where beta represents the investment's sensitivity to market movements

In [9]:
df = df_returns.copy()
    
def calculate_treynor_ratio(returns, market_returns, risk_free_rate):
    excess_return = returns - risk_free_rate
    beta = calculate_beta(returns, market_returns)

    if beta == 0:
        return np.inf  # Avoid division by zero

    treynor_ratio = excess_return.mean() / beta
    return treynor_ratio

print(f'{symbol_1} has a Treynor ratio of {calculate_treynor_ratio(df[symbol_1], df[market], 0)} used {market} as the market')
print(f'{symbol_2} has a Treynor ratio of {calculate_treynor_ratio(df[symbol_2], df[market], 0)} used {market} as the market')

AAPL has a Treynor ratio of 0.0016612994409675973 used ^GSPC as the market
TSLA has a Treynor ratio of 0.0017540341950601794 used ^GSPC as the market


## Calmar Ratio:

The Calmar Ratio is a risk-adjusted performance measure that evaluates the ratio of the average annual rate of return to the maximum drawdown, providing a measure of risk-adjusted returns. Higher Calmar ratios generally indicate better risk-adjusted performance.

In [10]:
df = df_returns.copy()
    
def calculate_calmar_ratio(returns, period=252):  # Assuming 252 trading days in a year
    annualized_return = np.mean(returns) * period
    max_drawdown = calculate_max_drawdown(returns)

    if max_drawdown == 0:
        return np.inf  # Avoid division by zero

    calmar_ratio = annualized_return / abs(max_drawdown)
    return calmar_ratio

print(f'{symbol_1} has a Calmar ratio of {calculate_calmar_ratio(df[symbol_1])}')
print(f'{symbol_2} has a Calmar ratio of {calculate_calmar_ratio(df[symbol_2])}')
print(f'{market} has a Calmar ratio of {calculate_calmar_ratio(df[market])}')

AAPL has a Calmar ratio of 1.2415328313451997
TSLA has a Calmar ratio of 1.5724403811854653
^GSPC has a Calmar ratio of 1.1393282311725124


## Ulcer Index:

The Ulcer Index is a measure of the depth and duration of drawdowns in an investment. It quantifies the extent of the price decline from its most recent peak, with lower values indicating less severe and shorter drawdowns

In [11]:
df = df_returns.copy()

def calculate_ulcer_index(returns):
    returns = returns.dropna()
    wealth_index = np.cumprod(1 + returns)
    previous_peaks = np.maximum.accumulate(wealth_index)
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    squared_drawdowns = np.square(drawdowns)
    ulcer_index = np.sqrt(np.mean(squared_drawdowns))
    return ulcer_index

print(f'{symbol_1} has an Ulcer index of {calculate_ulcer_index(df[symbol_1])}')
print(f'{symbol_2} has an Ulcer index of {calculate_ulcer_index(df[symbol_2])}')
print(f'{market} has an Ulcer index of {calculate_ulcer_index(df[market])}')



AAPL has an Ulcer index of 0.05509671290767418
TSLA has an Ulcer index of 0.14917579484097326
^GSPC has an Ulcer index of 0.03408355669692506


## Value at Risk (VaR):

Value at Risk (VaR) is a statistical measure used to quantify the potential loss on an investment over a specific time horizon and with a certain confidence level. There are different methods to calculate VaR, and one common approach is the historical simulation method.

In [12]:
df = df_returns.copy()
    
def calculate_var(returns, confidence_level=0.95):
    returns_sorted = np.sort(returns)
    n = len(returns)
    position = int(n * (1 - confidence_level))
    
    var = -returns_sorted[position]
    return var

print(f'{symbol_1} has a Value at Risk of {calculate_var(df[symbol_1])}')
print(f'{symbol_2} has a Value at Risk of {calculate_var(df[symbol_2])}')
print(f'{market} has a Value at Risk of {calculate_var(df[market])}')

AAPL has a Value at Risk of 0.01725376628542985
TSLA has a Value at Risk of 0.05030873801899527
^GSPC has a Value at Risk of 0.013788741489130674


## Conditional Value at Risk (CVaR):

Conditional Value at Risk (CVaR), also known as Expected Shortfall (ES), is a risk measure that quantifies the expected loss beyond the Value at Risk (VaR) at a certain confidence level. While VaR gives the maximum loss with a certain probability, CVaR provides an estimate of the expected loss given that the loss exceeds the VaR.

In [13]:
df = df_returns.copy()
    
def calculate_cvar(returns, confidence_level=0.95):
    sorted_returns = np.sort(returns)
    n = len(returns)
    position = int(n * (1 - confidence_level))

    cvar = -np.mean(sorted_returns[:position])
    return cvar

print(f'{symbol_1} has a Conditional Value at Risk of {calculate_cvar(df[symbol_1])}')
print(f'{symbol_2} has a Conditional Value at Risk of {calculate_cvar(df[symbol_2])}')
print(f'{market} has a Conditional Value at Risk of {calculate_cvar(df[market])}')

AAPL has a Conditional Value at Risk of 0.025631181708822726
TSLA has a Conditional Value at Risk of 0.06672154043052804
^GSPC has a Conditional Value at Risk of 0.015845102985801485


## Downside Deviation:

Downside Deviation is a risk metric that focuses on the volatility of negative returns or downside risk. It measures the dispersion of returns below a certain threshold, typically zero.

In [14]:
df = df_returns.copy()
    
def calculate_downside_deviation(returns, threshold=0):
    downside_returns = np.minimum(returns - threshold, 0)
    downside_deviation = np.std(downside_returns)
    return downside_deviation

print(f'{symbol_1} has a Downside Deviation of {calculate_downside_deviation(df[symbol_1])}')
print(f'{symbol_2} has a Downside Deviation of {calculate_downside_deviation(df[symbol_2])}')
print(f'{market} has a Downside Deviation of {calculate_downside_deviation(df[market])}')

AAPL has a Downside Deviation of 0.006788309680065321
TSLA has a Downside Deviation of 0.01770799377974175
^GSPC has a Downside Deviation of 0.004562120690564971


## R-Squared:

R-squared, also known as the coefficient of determination, is a statistical measure that represents the proportion of the variance in the dependent variable that is explained by the independent variable(s) in a regression model. In the context of financial analysis, it can be used to assess how well the performance of an investment or portfolio is explained by a benchmark index or other factors.


In [15]:
df = df_returns.copy()
    
def calculate_r_squared(actual, predicted):
    mean_actual = np.mean(actual)
    total_sum_of_squares = np.sum((actual - mean_actual) ** 2)
    residual_sum_of_squares = np.sum((actual - predicted) ** 2)
    
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)
    return r_squared

print(f'{symbol_1} has a R-Squared of {calculate_r_squared(df[symbol_1], df[market])} using {market} as the benchmark index')
print(f'{symbol_2} has a R-Squared of {calculate_r_squared(df[symbol_2], df[market])} using {market} as the benchmark index')



AAPL has a R-Squared of 0.5165675266526513 using ^GSPC as the benchmark index
TSLA has a R-Squared of 0.20515035436500972 using ^GSPC as the benchmark index
